In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet50, ResNet50V2, ResNet101, ResNet101V2, ResNet152, ResNet152V2,
    InceptionV3, InceptionResNetV2, Xception,
    MobileNet, MobileNetV2,
    DenseNet121, DenseNet169, DenseNet201,
    NASNetMobile
)
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnetv2_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as inceptionresnet_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications.nasnet import preprocess_input as nasnet_preprocess

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Set seeds for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Constants
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
BATCH_SIZE = 32
EPOCHS = 64
IMG_SIZE = (224, 224)

# Data Generators
def get_generators(preprocess_func):
    train_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func,
        horizontal_flip=True,
        zoom_range=0.2,
        rotation_range=15
    ).flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        seed=SEED
    )

    test_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func
    ).flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    return train_gen, test_gen

# Build classifier head
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

# Models Dictionary
models = {
    'Xception': (Xception(weights='imagenet', include_top=False, input_shape=(224,224,3)), xception_preprocess),
    'VGG16': (VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)), vgg_preprocess),
    'VGG19': (VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3)), vgg19_preprocess),
    'ResNet50': (ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnet_preprocess),
    'ResNet50V2': (ResNet50V2(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnetv2_preprocess),
    'ResNet101': (ResNet101(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnet_preprocess),
    'ResNet101V2': (ResNet101V2(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnetv2_preprocess),
    'ResNet152': (ResNet152(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnet_preprocess),
    'ResNet152V2': (ResNet152V2(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnetv2_preprocess),
    'InceptionV3': (InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3)), inception_preprocess),
    'InceptionResNetV2': (InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3)), inceptionresnet_preprocess),
    'MobileNet': (MobileNet(weights='imagenet', include_top=False, input_shape=(224,224,3)), mobilenet_preprocess),
    'MobileNetV2': (MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3)), mobilenetv2_preprocess),
    'DenseNet121': (DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3)), densenet_preprocess),
    'DenseNet169': (DenseNet169(weights='imagenet', include_top=False, input_shape=(224,224,3)), densenet_preprocess),
    'DenseNet201': (DenseNet201(weights='imagenet', include_top=False, input_shape=(224,224,3)), densenet_preprocess),
    'NASNetMobile': (NASNetMobile(weights='imagenet', include_top=False, input_shape=(224,224,3)), nasnet_preprocess),
}

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2)

# Result storage
report_rows = []

# Training and Evaluation Loop
for model_name, (base_model, preprocess_func) in models.items():
    print(f"\n🔧 Training model: {model_name}")
    train_gen, test_gen = get_generators(preprocess_func)
    model = build_model(base_model)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=test_gen,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    predictions = model.predict(test_gen, verbose=0)
    predicted_labels = (predictions > 0.5).astype(int).ravel()
    true_labels = test_gen.classes

    # Accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Classification Report
    report = classification_report(true_labels, predicted_labels, digits=6, output_dict=True)

    row = {
        'Model': model_name,
        'Accuracy': accuracy,
        'Macro Precision': report['macro avg']['precision'],
        'Macro Recall': report['macro avg']['recall'],
        'Macro F1': report['macro avg']['f1-score'],
        'Weighted Precision': report['weighted avg']['precision'],
        'Weighted Recall': report['weighted avg']['recall'],
        'Weighted F1': report['weighted avg']['f1-score'],
    }

    report_rows.append(row)

# Save to CSV
df_report = pd.DataFrame(report_rows)
df_report.to_csv('/kaggle/working/classification_summary.csv', index=False)
print("✅ Saved summary to /kaggle/working/classification_summary.csv")
print("📊 Classification Summary:")
print(df_report)


2025-07-17 14:39:53.205248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752763193.419566      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752763193.483057      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1752763207.335136      18 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

🔧 Training model: Xception
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64


I0000 00:00:1752763309.247978      56 service.cc:148] XLA service 0x7e0eb803c3b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752763309.248897      56 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752763310.221013      56 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1752763314.994331      56 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/64
163/163 - 165s - 1s/step - accuracy: 0.9141 - loss: 0.2178 - val_accuracy: 0.9054 - val_loss: 0.2513 - learning_rate: 0.0010
Epoch 2/64
163/163 - 96s - 592ms/step - accuracy: 0.9571 - loss: 0.1100 - val_accuracy: 0.8734 - val_loss: 0.3015 - learning_rate: 0.0010
Epoch 3/64
163/163 - 96s - 586ms/step - accuracy: 0.9663 - loss: 0.0956 - val_accuracy: 0.9167 - val_loss: 0.2296 - learning_rate: 0.0010
Epoch 4/64
163/163 - 96s - 587ms/step - accuracy: 0.9628 - loss: 0.0918 - val_accuracy: 0.8862 - val_loss: 0.3191 - learning_rate: 0.0010
Epoch 5/64
163/163 - 96s - 587ms/step - accuracy: 0.9613 - loss: 0.0949 - val_accuracy: 0.8718 - val_loss: 0.3432 - learning_rate: 0.0010
Epoch 6/64
163/163 - 98s - 601ms/step - accuracy: 0.9743 - loss: 0.0691 - val_accuracy: 0.8798 - val_loss: 0.3282 - learning_rate: 2.0000e-04

🔧 Training model: DenseNet201
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
163/163 - 180s - 1s/step - accuracy: 0.9045 - loss: 0.2289 - val_accuracy: 0.8253 - val_loss: 0.4366 - learning_rate: 0.0010
Epoch 2/64
163/163 - 96s - 591ms/step - accuracy: 0.9601 - loss: 0.1101 - val_accuracy: 0.8542 - val_loss: 0.3458 - learning_rate: 0.0010
Epoch 3/64
163/163 - 96s - 591ms/step - accuracy: 0.9603 - loss: 0.1033 - val_accuracy: 0.8622 - val_loss: 0.3595 - learning_rate: 0.0010
Epoch 4/64
163/163 - 95s - 586ms/step - accuracy: 0.9651 - loss: 0.0948 - val_accuracy: 0.8253 - val_loss: 0.5235 - learning_rate: 0.0010
Epoch 5/64
163/163 - 95s - 583ms/step - accuracy: 0.9714 - loss: 0.0739 - val_accuracy: 0.8606 - val_loss: 0.3888 - learning_rate: 2.0000e-04

🔧 Training model: NASNetMobile
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
163/163 - 163s - 1s/step - accuracy: 0.8972 - loss: 0.2462 - val_accuracy: 0.8077 - val_loss: 0.4384 - learning_rate: 0.0010
Epoch 2/64
163/163 - 93s - 570ms/step - accuracy: 0.9277 - loss: 0.1775 - val_accuracy: 0.8750 - val_loss: 0.3256 - learning_rate: 0.0010
Epoch 3/64
163/163 - 93s - 571ms/step - accuracy: 0.9402 - loss: 0.1572 - val_accuracy: 0.8734 - val_loss: 0.3480 - learning_rate: 0.0010
Epoch 4/64
163/163 - 94s - 574ms/step - accuracy: 0.9463 - loss: 0.1413 - val_accuracy: 0.8141 - val_loss: 0.4751 - learning_rate: 0.0010
Epoch 5/64
163/163 - 94s - 576ms/step - accuracy: 0.9484 - loss: 0.1242 - val_accuracy: 0.8590 - val_loss: 0.3592 - learning_rate: 2.0000e-04
✅ Saved summary to /kaggle/working/classification_summary.csv
📊 Classification Summary:
                Model  Accuracy  Macro Precision  Macro Recall  Macro F1  \
0            Xception  0.884615         0.885817      0.865812  0.873887   
1               VGG16  0.899038         0.894887      0.888462  0.89